In [1]:
import os
import logging
logging.getLogger().setLevel("DEBUG")

In [2]:
from keras.models import load_model

Using TensorFlow backend.


In [3]:
from models import ReverseComplementLayer
from models import CRF
from models import CRF_ext

In [4]:
import os
import logging
logger = logging.getLogger()
logger.setLevel('DEBUG')
# config for server
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
os.environ['PATH'] = r'C:\Users\Rudolf\Documents\v9.0\bin' + os.path.pathsep + os.environ['PATH'] 
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [5]:
import tensorflow as tf
from keras import backend as K
num_CPU = 2
num_GPU = 0
config = tf.ConfigProto( device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
session = tf.Session(config=config)
K.set_session(session)

# Utilty functions

In [6]:
import time
import datetime
def func_metrics_display(funk):
    def metricized(*args, **kwargs):
        start = time.time()
        obj = funk(*args, **kwargs)
        end = time.time()
        dt = end - start
        if dt > 1:
            format_str = ('%H hours' if dt >= 3600 else '') + ('%M minutes' if dt >= 60 else '') + '%S seconds'
            logging.info('The function {} took {}'.format(funk.__name__, 
                                                          time.strftime( format_str ,time.gmtime(dt)) ))
        else:
            logging.info('The function {} took {:>10.4f}s'.format(funk.__name__, dt))
        return obj
    metricized.__name__=funk.__name__
    return metricized

In [7]:
class InsufficientChromosomesException(Exception):
    def __init__(self, chrom_avail, chrom_req, msg=None):
        self.chrom_avail = chrom_avail
        self.chrom_req = chrom_req
        if not msg:
            msg = 'Found {} chromosomes. Needed more than {} chromosomes.'.format(chrom_avail, chrom_req)
        super(InsufficientChromosomesException, self).__init__(msg)

In [8]:
def is_in(a, domain):
    return domain[0]<=a<=domain[1]

def is_in_domain(peak, domain):
    '''
    Checks to see if peak (tuple of ints) overlaps with domain (tuple of ints) 
    '''
    p_start, p_end = peak
    d_start, d_end = domain
    return is_in(p_start, domain) or is_in(p_end, domain) or (is_in(d_start, peak) or is_in(d_end, peak))

def any_in_domain(peaks, domain):
    for peak in peaks:
        if is_in_domain(peak, domain):
            return True
    return False

def which_in_domain(peaks,domain):
    for peak in peaks:
        if is_in_domain(peak,domain):
            return peak
    return None

def domain2seq(chromosomes, chromosome, domain):
    return chromosomes[chromosome][domain[0]:domain[1]]

def where_in_domain(chromosomes, chromosome, peak, domain):
    peak_start, peak_end = peak
    domain_start, domain_end = domain
    
    seq = domain2seq(chromosomes, chromosome, domain)

    labels = []
    for i in range(domain_end-domain_start):
        dna_location_ptr = domain_start+i
        if dna_location_ptr<peak_start:
            labels.append('O')
        elif dna_location_ptr==peak_start:
            labels.append('B')
        elif peak_start<dna_location_ptr<peak_end:
            labels.append('I')
        elif dna_location_ptr==peak_end:
            labels.append('E')
        else:
            labels.append('O')
    return ''.join(labels), (chromosome, domain)

In [9]:
@func_metrics_display
def if_not_pickled(pkl_file, generator,log=True,gen_name=None):
    try:
        name = gen_name if gen_name else generator.__name__
    except AttributeError:
        name = generator.func.__name__
        
    def generate_pkl():
        obj = generator()
        with open(pkl_file, 'wb') as pkl:
            pickle.dump(obj, pkl)
        return obj
    
    if not os.path.exists(pkl_file):
        if log:
            logging.info('No pickle for {} is found. Generating anew.'.format(name))               
        obj = generate_pkl()
    else:
        if log:
            logging.info('Loading pickled {}'.format(name))
        try:
            with open(pkl_file, 'rb') as pkl:
                obj = pickle.load(pkl)
        except (EOFError,pickle.UnpicklingError) as e :
            logging.error('A pickle file was corrupted: {}'.format(pkl_file))
            if log:
                logging.info('Regenerating corrupted pickle: {}'.format(pkl_file))
            obj = generate_pkl()
    if log:
        logging.info('Finished setting up {}'.format(name))
    return obj

In [10]:
@func_metrics_display
def gen_chr2locNbound(label_file, _cellline, log=True):
    from tqdm import tqdm
    chr2locNbound = {}
    with open(label_file) as labels:
        line_gen = (line for line in labels)
        column_names = next(line_gen).strip().split() 
        # columns names are chr start stop <cell line 1> ... <cell line n>
        _, _, _, *celllines = column_names
        prev_chrom = None
        for line in tqdm(line_gen, total=get_num_lines(label_file)):
            chromosome, start, stop, *bound_per_cellline = [x.strip() for x in line.strip().split()]
            try:
                start,stop = [int(x) for x in [start,stop]]
            except ValueError:
                # ill formatted entry
                continue
            if log and prev_chrom!=chromosome:
                prev_chrom=chromosome
                logging.info('Working on {}\n'.format(chromosome))
            for cellline, is_bound in zip(celllines, bound_per_cellline):
                if cellline.lower() != _cellline.lower():
                    continue
                try:
                    chr2locNbound[chromosome].append(((start,stop), is_bound))
                except KeyError:
                    chr2locNbound[chromosome] = [((start,stop), is_bound)]                
    return chr2locNbound

In [11]:
@func_metrics_display
def gen_hg19(hg_genome_fasta):
    chromosomes = {}
    with open(hg_genome_fasta) as hg19:
        chromosome = None
        for line in hg19:
            if line.startswith('>'):
                chromosome = line[1:].strip()
                chromosomes[chromosome] = []
            else: 
                chromosomes[chromosome].append(line.strip().upper())

    for k,v in chromosomes.items():
        chromosomes[k] = ''.join(v)   
    return chromosomes

In [12]:
@func_metrics_display
def gen_chr2filter_locs(filter_file):
    chr2filter_locs = {}
    with open(filter_file) as filter_f:
        for line in filter_f:
            chromosome, start, end = line.strip().split()
            start, end = [int(x) for x in (start, end)]
            try:
                chr2filter_locs[chromosome].append((start,end))
            except KeyError:
                chr2filter_locs[chromosome] = [(start,end)]
    return chr2filter_locs

In [13]:
@func_metrics_display
def gen_chr2locNpeaks(celllineNtf_peakfile, filter_file=None, chr2filter_locs=None):
    chr2locNpeaks = {}
    from tqdm import tqdm
    with open(celllineNtf_peakfile) as peaks:
        for line in tqdm(peaks, total=get_num_lines(celllineNtf_peakfile)):
            chromosome, start, stop, name, score, strand, signal, p, q, peak = line.strip().split()
            start, stop = [int(x) for x in [start,stop]]
            try:
                chr2locNpeaks[chromosome]
            except KeyError:
                chr2locNpeaks[chromosome]=[]
                
            if filter_file:
                if not chr2filter_locs:
                    raise ValueError('Need chr2filter_locs if using filter_file')
                filter_locs = chr2filter_locs[chromosome]
                overlap=False
                for loc in chr2filter_locs[chromosome]:
                    if loc[0]<=start<=loc[1] or loc[0]<=stop<=loc[1]:
                        overlap=True
                        break
                if not overlap:
                    continue
            chr2locNpeaks[chromosome].append((start,
                                              stop,
                                             {'name' : name,
                                             'score': int(score),
                                             'strand': strand,
                                             'p-value':float(p),
                                             'q-value':float(q),
                                             'peak':int(peak)}))
        for chromosome, lst in chr2locNpeaks.items():
            chr2locNpeaks[chromosome] = sorted(lst,key= lambda x:x[0])
    
    return chr2locNpeaks

In [14]:
def seq2sequence(seq, chromosomes):
    chromosome=seq[0]
    domain=seq[1]
    return domain2seq(chromosomes, chromosome, domain)

@func_metrics_display
def gen_chr2labelsNseq(chromosomes, chr2locNbound, chr2locNpeaks):
    chr2labelsNseq = {}
    from tqdm import tqdm
    shared_chromosomes = set(chr2locNbound.keys()) & set(chr2locNpeaks.keys())
    logging.info('Generating labels and seq for this set of chromosomes {}'.format(str(shared_chromosomes)))
    for chromosome in tqdm(shared_chromosomes):
        logging.info('Generating labels and seq for {}'.format(chromosome))
        bound_locs = [x for x in chr2locNbound[chromosome] if x[1]=='B']
        bound_locs = [loc[0] for loc in bound_locs]
        peaks_locs = [x[:2] for x in chr2locNpeaks[chromosome]]
        
#         peak_offsets = [x[2]['peak'] for x in chr2locNpeaks[chromosome]]
        count = 0
        for bound_loc in bound_locs:
            # generate peak information
            the_peak = which_in_domain(peaks_locs, bound_loc)
            if not the_peak:
                logging.warning('Cannot find peak {}'.format(bound_loc))
                continue
            the_peak_i = peaks_locs.index(the_peak)
            labels, seq = where_in_domain(chromosomes,
                                          chromosome,
                                          the_peak,
                                          bound_loc)
            if not ('B' in labels or 'I' in labels or 'E' in labels):
                raise ValueError('BIE not found in labels\nsequence:{}\nlabels{}'.format(seq,labels))
            try:
                chr2labelsNseq[chromosome].append((labels, seq))
            except KeyError:
                chr2labelsNseq[chromosome] = [(labels, seq)]
            count += 1
        logging.info('Found {}/{} seqs for bound locations'.format(count, len(bound_locs)))        
        unbound_locs = [x for x in chr2locNbound[chromosome] if x[1]=='U']
        unbound_locs = [loc[0] for loc in unbound_locs]
        count = 0
        for unbound_loc in unbound_locs:
            seq = (chromosome, unbound_loc)
            try:
                chr2labelsNseq[chromosome].append((None, seq))
            except KeyError:
                chr2labelsNseq[chromosome] = [(None, seq)]
            count += 1
        logging.info('Found {}/{} seqs for bound locations'.format(count, len(unbound_locs)))  
    return chr2labelsNseq

In [15]:
def read_structure_file(filename):
    with open(filename, 'r') as f:
        lines = []
        agg = []
        for line in f:
            if line.startswith('>'):
                lines.append(','.join(agg))
                agg = []
            else:
                agg.append(line.strip())
        lines.append(','.join(agg))
    del lines[0]
    return lines

In [16]:
def map_paths_exist(*paths):
    import os
    return [os.path.exists(path) for path in paths]

def all_paths_exists(*paths):
    from functools import reduce
    return reduce(lambda acc,x: acc and x ,map_paths_exist(*paths))

In [17]:
import mmap
def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [18]:
instanceHolder = {"instance": None}
class ClassWrapper(CRF_ext):
    def __init__(self, *args, **kwargs):
        instanceHolder["instance"] = self
        super(ClassWrapper, self).__init__(*args, **kwargs)
def loss(*args):
    method = getattr(instanceHolder["instance"], "loss_function")
    return method(*args)
def accuracy(*args):
    method = getattr(instanceHolder["instance"], "accuracy")
    return method(*args)
def viterbi_precision(*args):
    method = getattr(instanceHolder["instance"], "viterbi_precision")
    return method(*args)
def f1(*args):
    method = getattr(instanceHolder["instance"], "viterbi_f1")
    return method(*args)
def recall(*args):
    method = getattr(instanceHolder["instance"], "viterbi_recall")
    return method(*args)
def precision(*args):
    method = getattr(instanceHolder["instance"], "viterbi_precision")
    return method(*args)

In [19]:
import os
import pickle
import logging
from functools import partial 
import tqdm
import numpy as np

_cache = {}
class DataManager:
    
    def __init__(self,label_file, celllineNtf_peakfile, 
                 bigwig_duke_unique_file = './wgEncodeDukeMapabilityUniqueness35bp.bigWig',
                 bigwig_dnase_file = None,
                 filter_file=None,
                 use_pickler=False, memory_avail=8192, 
                 output_dir='./',
                 only_label_dataset = True,
                 move_finished_src = None, 
                 reduce_negative_samples=True, 
                 chr_valid = ['chr11'], chr_test = ['chr1', 'chr8', 'chr21'], 
                 check_set_ratio=9, debug={}):
        
        self.label_file = label_file
        self.celllineNtf_peakfile = celllineNtf_peakfile
        self.bigwig_duke_unique_file = bigwig_duke_unique_file
        self.duke_bw = bigwig_duke_unique_file
        self.use_pickler = use_pickler
        self.memory_avail=memory_avail
        self.output_dir = output_dir
        self.only_label_dataset=only_label_dataset
        self.move_finished_src = move_finished_src
        self.filter_file = filter_file
        self.reduce_negative_samples = reduce_negative_samples
        self.chr_valid = chr_valid
        self.chr_test = chr_test
        self.check_set_ratio = check_set_ratio
        self.debug = debug
        self.exp_type, self._cellline, self.tf_name,_, self.set_name, self.peak_type = os.path.basename(celllineNtf_peakfile).split('.')
        self.dnase_bw = bigwig_dnase_file if bigwig_dnase_file else '%s.1x.bw' % self._cellline.lower()
        self.exp_id = '{}_{}'.format(self.tf_name, self._cellline)
#         self.bigwig_cellline_file = get_bigwig_celline_file(self._cellline)
        self.positive_samples_out, self.negative_samples_out, self.labels_out = [self.exp_id+'_'+x for x in ['positive_samples.txt', 'negative_samples.txt', 'labels.txt']]
        self.chr_positive_out, self.chr_negative_out = [self.exp_id+'_chr_%s.npy' % sign for sign in ['positive', 'negative']]

    def _cacher_(func):
        def to_cache_func(self, *args, **kwargs):
            if func.__name__ in _cache:
                return _cache[func.__name__]
            obj = func(self, *args, **kwargs)
            _cache[func.__name__] = obj
            return obj
        return to_cache_func
    
    def release_memory(self):
        import gc
        _cache = {}
        gc.collect()
    
    @_cacher_
    def _chr2locNbound(self):
        chr2locNbound_pklfile = '{}_chr2locNbound.pkl'.format(self.exp_id)
        pgen_chr2locNbound = partial(gen_chr2locNbound, self.label_file, self._cellline)
        chr2locNbound = if_not_pickled(chr2locNbound_pklfile,
                                       pgen_chr2locNbound)
        return chr2locNbound
    @property
    def chr2locNbound(self):
        return self._chr2locNbound()
    
    @_cacher_
    def _chr2filter_locs(self):
        chr2filter_locs_pklfile = '{}_chr2filter_locs.pkl'.format(self.exp_id)
        pgen_chr2filter_locs = partial(gen_chr2filter_locs, 
                                       self.filter_file)
        chr2filter_locs = if_not_pickled(chr2filter_locs_pklfile, 
                                         pgen_chr2filter_locs)
        return chr2filter_locs
    @property
    def chr2filter_locs(self):
        return self._chr2filter_locs()
    
    @_cacher_
    def _chr2locNpeaks(self):
        chr2locNpeaks_pklfile = '{}_chr2locNpeaks.pkl'.format(self.exp_id) if self.filter_file else '{}_chr2locNpeaks_full.pkl'.format(self.exp_id)
        p_genchr2locNpeaks = partial(gen_chr2locNpeaks, 
                                     self.celllineNtf_peakfile, 
                                     self.filter_file, 
                                     self.chr2filter_locs if self.filter_file else None)
        chr2locNpeaks = if_not_pickled(chr2locNpeaks_pklfile,
                                       p_genchr2locNpeaks)
        return chr2locNpeaks
    @property
    def chr2locNpeaks(self):
        return self._chr2locNpeaks()
    
    @_cacher_
    def _chromosomes(self):
        hg_pkl = 'hg19.pkl'
        hg_genome_fasta = './hg19.genome.fa'
        pgen_hg19 = partial(gen_hg19, hg_genome_fasta)
        chromosomes = if_not_pickled(hg_pkl, pgen_hg19)
        return chromosomes
    @property
    def chromosomes(self):
        return self._chromosomes()
    
    @_cacher_
    def _chr2labelsNseq(self):
        chr2labelsNseq_pkl = '{}_chr2labelsNseq.pkl'.format(self.exp_id)
        pgen_chr2labelsNseq = partial(gen_chr2labelsNseq,
                                      self.chromosomes,
                                      self.chr2locNbound, 
                                      self.chr2locNpeaks)
        chr2labelsNseq = if_not_pickled(chr2labelsNseq_pkl, 
                                        pgen_chr2labelsNseq)
        del _cache['_chr2locNpeaks']
        del _cache['_chr2locNbound']
        del _cache['_chr2filter_locs']
        return chr2labelsNseq
    @property
    def chr2labelsNseq(self):
        return self._chr2labelsNseq()
    
    def _create_samples(self):
        
        logging.info('Creating positive/negative samples')
        # generate samples
        positive_samples, negative_samples, labels_positives = [],[],[]
        pos_chr, neg_chr = [], [] # keep track of which chromosome each are from
        
        chr_training = set(self.chr2labelsNseq.keys()) - set(self.chr_valid) - set(self.chr_test)
        if not len(chr_training):
            raise InsufficientChromosomesException(set(self.chr2labelsNseq.keys()), set(self.chr_valid) | set(self.chr_test))
        from tqdm import tqdm
        for chromosome, labelsNseq in tqdm(self.chr2labelsNseq.items()):
            _positive_samples,_negative_samples, _labels_positives = [],[],[]
            _pos_chr, _neg_chr = [], []
            logging.info('Working on {}'.format(chromosome))
            count = 0
            pos_count = 0
            neg_count = 0
            for label, seq in labelsNseq:
                # labels is None if negative sample
                sequence = seq2sequence(seq, self.chromosomes)
                if 'N' not in sequence:
                    if label==None:
                        _negative_samples.append(sequence)
                        _neg_chr.append(seq)
                        neg_count += 1
                    else:
                        _positive_samples.append(sequence)
                        _labels_positives.append(label)
                        _pos_chr.append(seq)
                        pos_count += 1
                    count += 1
            logging.info('Found {} samples for {}; {} positive and {} negative'.format(count, 
                                                                                       chromosome, 
                                                                                       pos_count, 
                                                                                       neg_count))
            # throw away some negative samples
#             if self.reduce_negative_samples :
#                 logging.info('Reducing negative samples for {}'.format(chromosome))
#                 ratio = 1 if not (chromosome in self.chr_valid or chromosome in self.chr_test) else self.check_set_ratio
#                 if len(_negative_samples) > len(_positive_samples):
#                     npified = np.array([_negative_samples, _neg_chr], dtype=np.object)
#                     n_to_choose_from = ratio * len(_positive_samples)
#                     if n_to_choose_from > len(_negative_samples):
#                         n_to_choose_from = len(_negative_samples)
#                     indices_chosen = np.random.choice(len(_negative_samples), 
#                                                       n_to_choose_from, 
#                                                       replace=False)
#                     chosen_samples, chosen_chr = npified[:,indices_chosen]
#                     _negative_samples, _neg_chr = list(chosen_samples), list(chosen_chr)
            
            logging.info('Found {} samples for {}; {} positive and {} negative'.format(count, 
                                                                                       chromosome, 
                                                                                       len(_positive_samples), 
                                                                                       len(_negative_samples)))
            positive_samples.extend(_positive_samples)
            negative_samples.extend(_negative_samples)
            labels_positives.extend(_labels_positives)
            pos_chr.extend(_pos_chr)
            neg_chr.extend(_neg_chr)
            
        # write samples to disk
        logging.info('Writing samples to disk')
        def write_to_disk(path, samples):
            with open(path, 'w') as samples_file:
                for i,sample in enumerate(samples):
                    samples_file.write('>{}__{}\n'.format(self.tf_name, i))
                    samples_file.write('{}\n'.format(sample))
        
        assert len(positive_samples) == len(pos_chr) == len(labels_positives)
        assert len(negative_samples) == len(neg_chr)
        write_to_disk(self.positive_samples_out, positive_samples)
        write_to_disk(self.negative_samples_out, negative_samples)
        write_to_disk(self.labels_out, labels_positives)
        np.save(self.chr_positive_out, np.array(pos_chr, dtype=np.object))
        np.save(self.chr_negative_out, np.array(neg_chr, dtype=np.object))
        logging.info('Done writing samples to disk')
        self._positive_samples, self._negative_samples, self._labels_positives = positive_samples, negative_samples, labels_positives
        self._pos_chr, self._neg_chr = pos_chr, neg_chr
        return positive_samples, negative_samples, labels_positives, pos_chr, neg_chr
    
    def _load_samples(self):
        # load samples from disk
        logging.info('Loading positive/negative samples from disk')
        positive_samples, negative_samples, labels_positives = [], [], []
        with open(self.positive_samples_out) as samples_file:
            for line in samples_file:
                if not line.startswith('>'):
                    positive_samples.append(line.strip())
        with open(self.negative_samples_out) as samples_file:
            for line in samples_file:
                if not line.startswith('>'):
                    negative_samples.append(line.strip())
        with open(self.labels_out) as samples_file:
            for line in samples_file:
                if not line.startswith('>'):
                    labels_positives.append(line.strip())  
        pos_chr = np.load(self.chr_positive_out)
        neg_chr = np.load(self.chr_negative_out) 
        logging.info('Done loading samples from disk')
        self._positive_samples, self._negative_samples, self._labels_positives, = positive_samples, negative_samples, labels_positives
        self._pos_chr, self._neg_chr = pos_chr, neg_chr
        return positive_samples, negative_samples, labels_positives, pos_chr, neg_chr
    
    @_cacher_
    def _samples(self):
        sample_files = [self.positive_samples_out, 
                        self.negative_samples_out, 
                        self.labels_out, 
                        self.chr_positive_out, 
                        self.chr_negative_out ]
        
        if not all_paths_exists(*sample_files):
            return self._create_samples()
        else: 
            return self._load_samples()
    @property
    def samples(self):
        return self._samples()
    
    @property
    def positive_samples(self):
        try:
            return self._positive_samples
        except:
            return self.samples[0]
    @property
    def negative_samples(self):
        try:
            return self._negative_samples
        except:
            return self.samples[1]
    @property
    def labels_positives(self):
        try:
            return self._labels_positives
        except:
            return self.samples[2]
    @property
    def pos_chr(self):
        try:
            return self._pos_chr
        except:
            return self.samples[3]
    @property
    def neg_chr(self):
        try:
            return self._neg_chr
        except:
            return self.samples[4]
    
    @func_metrics_display
    def dnashapeR(self, r_lib_location = "C:/Users/Rudolf/Documents/R/win-library/3.5"):
        # setup structural information
        # setup R
        import rpy2
        from rpy2.robjects.packages import importr
        import rpy2.robjects as robjects

        # set the available amount of memory
        robjects.r('memory.limit(size = {})'.format(self.memory_avail))

        base = importr('base')
        utils = importr('utils')
        logging.info('Using {}'.format(str(base._libPaths())))

        # if DNAshapeR cannot be found try this:
        robjects.r( ".libPaths('{}')".format(r_lib_location))

        from functools import reduce

        if not all_paths_exists(*[self.positive_samples_out+ext for ext in ['.EP', '.HelT', '.MGW', '.ProT', '.Roll']]):
            logging.info('Running DNAshapeR for positive samples')
            base = importr('base')
            utils = importr('utils')
            dna_shape = importr('DNAshapeR', lib_loc=r_lib_location)
            #rpy2 does not know how to release memory
            @rmem_manage
            def process_positive():
                r_statements = []
                r_statements.append('library(DNAshapeR)')
                r_statements.append('pred <- getShape("./{}")'.format(self.positive_samples_out))
                r_cmd = '\n'.join(r_statements)
                robjects.r(r_cmd)
            process_positive()
        else:
            logging.info('Skipping DNAshapeR for positive samples; already exists')
        if not all_paths_exists(*[self.negative_samples_out+ext for ext in ['.EP', '.HelT', '.MGW', '.ProT', '.Roll']]):
            logging.info('Running DNAshapeR for negative samples')
            base = importr('base')
            utils = importr('utils')
            dna_shape = importr('DNAshapeR', lib_loc=r_lib_location)
            gc.collect()
            @rmem_manage
            def process_negative():
                r_statements = []
                r_statements.append('library(DNAshapeR)')
                r_statements.append('pred <- getShape("./{}")'.format(self.negative_samples_out))
                r_cmd = '\n'.join(r_statements)
                robjects.r(r_cmd)
            process_negative()
        else:
            logging.info('Skipping DNAshapeR for negative samples; already exists')
    
    @_cacher_
    def _h5py_duke(self):
        return gen_h5py4bw(self.duke_bw, 'duke_unique')
    
    @property
    def h5py_duke(self):
        return self._h5py_duke()
    
    def duke_unique(self, chromosome, start, stop):
        return self.h5py_duke[chromosome][start:stop]
        
    @_cacher_
    def _h5py_dnase(self):
        return gen_h5py4bw(self.dnase_bw, self._cellline)
    
    @property
    def h5py_dnase(self):
        return self._h5py_dnase()
    
    def dnase(self, chromosome, start, stop):
        return self.h5py_dnase[chromosome][start:stop]
    
    def create_datagen_from_samples(self, balance_valid_ratio=9, useDNAshapeR=False):
        '''
        Returns functions that takes a batch_size as input and returns generators
        '''
        import pickle
        import numpy as np
        from tqdm import tqdm
        # files describing structure of dna
        exts = ['']
        if useDNAshapeR:
            raise NotImplementedError('DNAshapeR is not streamable for large datasets')
        dataset_pkl = '%s_dataset.pkl'%self.exp_id
        dataset_pkl = dataset_pkl.lower()
        import os
        import pickle
        if os.path.exists(dataset_pkl):
            logging.info('Reading static data')
            with open(dataset_pkl, 'rb') as pkl:
                train_positive_samples = pickle.load(pkl)
                train_negative_samples = pickle.load(pkl)
                train_labels_positives = pickle.load(pkl)
                train_pos_chr = pickle.load(pkl) 
                train_neg_chr = pickle.load(pkl)
                training_cls = pickle.load(pkl)
                training_labels = pickle.load(pkl)
                valid_samples = pickle.load(pkl)
                valid_chr = pickle.load(pkl)
                valid_cls = pickle.load(pkl)
                valid_labels = pickle.load(pkl)
                test_samples = pickle.load(pkl)
                test_chr = pickle.load(pkl)
                test_cls = pickle.load(pkl)
                test_labels = pickle.load(pkl)
                self.dataset2counts_pos = pickle.load(pkl)
                self.dataset2counts_neg = pickle.load(pkl)
                self.training_length = pickle.load(pkl)
                self.valid_length = pickle.load(pkl)
                self.test_length = pickle.load(pkl)
                self.sample_length = pickle.load(pkl)
                self.feature_dimensions = pickle.load(pkl)  
                logging.info('Loaded training/valid/test of sizes {}/{}/{}'.format(self.training_length, self.valid_length, self.test_length))
        else:     
            logging.info('Getting samples')
            # positive_samples and negative_samples are lists of sequences:str
            # pos_chr and neg_chr are lists of (chromosome:str, (start:int, stop:int))
            # labels_positives is a list of (for crf) labels:str
            positive_samples, negative_samples, labels_positives, pos_chr, neg_chr = self.samples

            logging.info('Shuffling samples')
            assert len(positive_samples) == len(labels_positives) == len(pos_chr)
            assert len(negative_samples) == len(neg_chr)

            pos_len = len(positive_samples)
            neg_len = len(negative_samples)

            class Pair:
                __slots__=['chromosome', 'domain']
                def __init__(self, chromosome, domain):
                    self.chromosome = chromosome
                    self.domain = domain
                def __repr__(self):
                    return '('+str(self.chromosome)+', '+str(self.domain)+')'
                def reveal(self):
                    return (self.chromosome, self.domain)

            @func_metrics_display
            def shuffle(*arrays):
                import numpy as np
                try:
                    npified = np.array(arrays, dtype=np.object)
                except Exception as e:
                    logging.error('{}'.format([np.array(a).shape for a in arrays]))
                    raise e
                length = npified.shape[1]
                indices_chosen = np.random.choice(length, length, replace=False)
                return npified[:,indices_chosen]

            pos_pairs = [Pair(*c) for c in pos_chr]
            neg_pairs = [Pair(*c) for c in neg_chr]

            positive_samples, labels_positives, pos_pairs = shuffle(positive_samples, labels_positives, pos_pairs)
            negative_samples, neg_pairs = shuffle(negative_samples, neg_pairs)

            pos_chr = [pair.reveal() for pair in pos_pairs]
            neg_chr = [pair.reveal() for pair in neg_pairs]

            # create stats on number of pos/neg for training/valid/test sets
            logging.info('Creating stats on number of pos/neg for training/valid/test sets')

            chr_valid_set = set(self.chr_valid)
            chr_test_set = set(self.chr_test)

            def try_addone(counts, key):
                try:
                    counts[key]+=1
                except KeyError:
                    counts[key]=1
            @func_metrics_display
            def count(chrs):            
                counts = {}   
                for chromosome, (start,stop) in chrs:
                    if chromosome in chr_valid_set:
                        try_addone(counts, 'valid')
                    elif chromosome in chr_test_set:
                        try_addone(counts, 'test')
                    else: # in training set
                        try_addone(counts, 'train')
                return counts

            pos_counts = count(pos_chr)
            neg_counts = count(neg_chr)
            logging.info('Counted positive for training/valid/test: {:>12}  {:>12}  {:>12}'.format(*[pos_counts[x]for x in ['train','valid','test']]))
            logging.info('Counted negative for training/valid/test: {:>12}  {:>12}  {:>12}'.format(*[neg_counts[x]for x in ['train','valid','test']]))
            self.dataset2counts_pos = pos_counts
            self.dataset2counts_neg = neg_counts

            logging.info('Splitting samples into training/valid/test')

            train_positive_samples, train_labels_positives, train_pos_chr = [], [], []
            valid_positive_samples, valid_labels_positives, valid_pos_chr = [], [], []
            test_positive_samples, test_labels_positives, test_pos_chr = [], [], []
            train_negative_samples, train_neg_chr = [], []
            valid_negative_samples, valid_neg_chr = [], []
            test_negative_samples, test_neg_chr = [], []

            logging.info('Preparing for positive training/valid/test')
            for sample, label, c in zip(positive_samples, labels_positives, pos_chr):
                chromosome, (start, stop) = c
                if chromosome in chr_valid_set:
                    valid_positive_samples.append(sample)
                    valid_labels_positives.append(label)
                    valid_pos_chr.append(c)
                elif chromosome in chr_test_set:
                    test_positive_samples.append(sample)
                    test_labels_positives.append(label)
                    test_pos_chr.append(c)
                else: # in training set
                    train_positive_samples.append(sample)
                    train_labels_positives.append(label)
                    train_pos_chr.append(c)
            logging.info('Preparing for negative training/valid/test')
            for sample, c in zip(tqdm(negative_samples), neg_chr):
                chromosome, (start, stop) = c
                if chromosome in chr_valid_set:
                    valid_negative_samples.append(sample)
                    valid_neg_chr.append(c)
                elif chromosome in chr_test_set:
                    test_negative_samples.append(sample)
                    test_neg_chr.append(c)
                else: # in training set
                    train_negative_samples.append(sample)
                    train_neg_chr.append(c)

            if self.use_pickler:
                logging.info('Setting up pickler to intercept used data')
                label_dataset = PickleManager('{}_label_dataset.pkl'.format(self.exp_id), overwrite=True)

            import numpy as np
            # start with positive samples
            onehot = {'A':(1.,0.,0.,0.),
                      'C':(0.,1.,0.,0.),
                      'G':(0.,0.,1.,0.),
                      'T':(0.,0.,0.,1.),}

            logging.info('Generating feature generators')
            # we want one for each pos/neg and train/valid/test
            # we want same number of pos/neg samples for training
            sample_start, sample_stop = train_pos_chr[0][1]
            sample_length = sample_stop-sample_start

            logging.info('Samples are found to have length {}'.format(sample_length))
            self.sample_length = sample_length
            self.feature_dimensions = 6

            logging.info('Generating static data for training')
            n_pos_train = len(train_positive_samples) 
            if self.reduce_negative_samples:
                train_negative_samples=train_negative_samples[:n_pos_train]
                train_neg_chr=train_neg_chr[:n_pos_train]
            n_neg_train = len(train_negative_samples)
            # shuffle the pos/neg for training
            # x, metadata, y, labels (for crf)
            training_samples, training_chr, training_cls, training_labels = shuffle(train_positive_samples + train_negative_samples,
                                                                                    train_pos_chr          + train_neg_chr,
                                                                                    [1]*n_pos_train        + [0]*n_neg_train,
                                                                                    train_labels_positives + ['O'*sample_length]*n_neg_train)
            
            self.dataset2counts_pos['train'] = n_pos_train
            self.dataset2counts_neg['train'] = n_neg_train
            self.training_length = n_pos_train*2
            logging.info('Counted positive for training/valid/test: {:>12}  {:>12}  {:>12}'.format(*[pos_counts[x]for x in ['train','valid','test']]))
            logging.info('Counted negative for training/valid/test: {:>12}  {:>12}  {:>12}'.format(*[neg_counts[x]for x in ['train','valid','test']]))

            logging.info('Generating static data for validation/test')
            # take all negative samples for valid/test instead of limiting it like in training
            if balance_valid_ratio:
                n_pos_valid = len(valid_positive_samples)
                valid_negative_samples = valid_negative_samples[:n_pos_valid*balance_valid_ratio]
                valid_neg_chr = valid_neg_chr[:n_pos_valid*balance_valid_ratio]
            self.valid_length = len(valid_positive_samples) + len(valid_negative_samples)
            self.test_length = len(test_positive_samples) + len(test_negative_samples)

            valid_samples, valid_chr, valid_cls, valid_labels = shuffle(valid_positive_samples          + valid_negative_samples,
                                                                        valid_pos_chr                   + valid_neg_chr,
                                                                        [1]*len(valid_positive_samples) + [0]*len(valid_negative_samples),
                                                                        valid_labels_positives          + ['O'*sample_length]*len(valid_negative_samples))

            test_samples, test_chr, test_cls, test_labels = shuffle(test_positive_samples          + test_negative_samples,
                                                                    test_pos_chr                   + test_neg_chr,
                                                                    [1]*len(test_positive_samples) + [0]*len(test_negative_samples),
                                                                    test_labels_positives          + ['O'*sample_length]*len(test_negative_samples))




            logging.info('Saving static data')
            with open(dataset_pkl, 'wb') as pkl:
                datalst= [train_positive_samples, train_negative_samples, train_labels_positives,
                          train_pos_chr, train_neg_chr,
                          training_cls, training_labels, 
                           valid_samples, valid_chr, valid_cls, valid_labels, 
                           test_samples, test_chr, test_cls, test_labels,
                          self.dataset2counts_pos, self.dataset2counts_neg, 
                          self.training_length, self.valid_length, self.test_length, self.sample_length,
                          self.feature_dimensions]
                for thing in tqdm(datalst):
                    pickle.dump(thing, pkl)
            
        logging.info('Creating generators')
    
        onehot = {'A':(1.,0.,0.,0.),
                  'C':(0.,1.,0.,0.),
                  'G':(0.,0.,1.,0.),
                  'T':(0.,0.,0.,1.),}
        def features_gen(samples, chrs):
            def batch_features_generator(batch_size):
                max_k = len(samples) // batch_size
                k = 0
                while True:
                    batch = []
                    
                    for i in range(batch_size):
                        seq, features = samples[i+k*batch_size], chrs[i+k*batch_size]
                        chromosome, domain = features
                        start, stop = domain
                        # this part is potentially very slow #
                        uniqueness = np.array(self.duke_unique(chromosome, start, stop))
                        openness = np.array(self.dnase(chromosome, start, stop))
                        uniqueness[np.isnan(uniqueness)] = 0
                        openness[np.isnan(openness)] = 0
                        # # # # # # # # # # # # # # # # # # # 
                        features = []
                        for i in range((stop-start)):
                            feature = []
                            feature.extend( onehot[seq[i]] )
                            feature.append( openness[i] )
                            feature.append( uniqueness[i] )
                            features.append(feature)
                        batch.append(np.array(features))
                    yield np.array(batch)
                    # cycle for infinite generator
                    k+=1
                    if k==max_k:
                        k=0
            return batch_features_generator
        
        logging.info('Creating feature generators')
#         train_x = features_gen(train_positive_samples + train_negative_samples, training_chr)
        valid_x = features_gen(valid_samples, valid_chr)
        test_x = features_gen(test_samples, test_chr)
        
        @func_metrics_display
        def values_gen(items, dtype=np.object):
            def batch_generator(batch_size):
                max_k = len(items) // batch_size
                k = 0
                while True:
                    result = [items[i+k*batch_size] for i in range(batch_size)]
                    yield np.array(result, dtype=dtype)
                    # cycle for infinite generator
                    k+=1
                    if k==max_k:
                        k=0
            return batch_generator
        
        logging.info('Calculating sample weights')
        def sample_weights(c, pos_count, neg_count):
            neg_weight = pos_count/neg_count
            return (1-c)*neg_weight + c        
        
        logging.info('Creating features sequencers')
        train_labels_sequencer = lambda batch_size: TrainingFeaturesSequence((train_positive_samples, train_negative_samples), 
                                                                             (train_pos_chr, train_neg_chr), 
                                                                             (train_labels_positives, None), 
                                                                     batch_size, 
                                                                     self.sample_length, self.feature_dimensions, 
                                                                     self.duke_unique, self.dnase)
        train_cls_sequencer = lambda batch_size: TrainingFeaturesSequence((train_positive_samples, train_negative_samples), 
                                                                          (train_pos_chr, train_neg_chr), 
                                                                          ([1]*self.dataset2counts_pos['train'] , None), 
                                                                     batch_size, 
                                                                     self.sample_length, self.feature_dimensions, 
                                                                     self.duke_unique, self.dnase)
        valid_labels_sequencer = lambda batch_size: FeaturesSequence(valid_samples, valid_chr, valid_labels, 
                                                                     batch_size, 
                                                                     self.sample_length, self.feature_dimensions, 
                                                                     self.duke_unique, self.dnase)
        valid_cls_sequencer = lambda batch_size: FeaturesSequence(valid_samples, valid_chr, valid_cls, 
                                                                     batch_size, 
                                                                     self.sample_length, self.feature_dimensions, 
                                                                     self.duke_unique, self.dnase)
        test_labels_sequencer = lambda batch_size: FeaturesSequence(test_samples, test_chr, test_labels, 
                                                                     batch_size, 
                                                                     self.sample_length, self.feature_dimensions, 
                                                                     self.duke_unique, self.dnase)
        test_cls_sequencer = lambda batch_size: FeaturesSequence(test_samples, test_chr, test_cls, 
                                                                    batch_size, 
                                                                    self.sample_length, self.feature_dimensions, 
                                                                    self.duke_unique, self.dnase)
        
        
        logging.info('Creating value generators')
        
#         train_label = values_gen(training_labels)
        valid_label = values_gen(valid_labels)
        test_label = values_gen(test_labels)
        
#         train_seq = values_gen(train_pos_chr+train_neg_chr)
        valid_seq = values_gen(valid_chr)
        test_seq = values_gen(test_chr)
        
#         train_y = values_gen(training_cls, int)
        valid_y = values_gen(valid_cls, int)
        test_y = values_gen(test_cls, int)

        return  (
#                 train_x, train_label, train_seq, train_y, 
                 train_labels_sequencer, train_cls_sequencer,
                 valid_x, valid_label, valid_seq, valid_y, valid_labels_sequencer, valid_cls_sequencer, 
                 test_x, test_label, test_seq, test_y, test_labels_sequencer, test_cls_sequencer)
        
    def load_data(self, *names):
        label_dataset_output_dir = os.path.join(self.output_dir, self.exp_id+'_label_dataset')  
        paths = [os.path.join(label_dataset_output_dir, name+'.npy') for name in names]
        return [np.load(path) for path in paths]

In [20]:
import keras
import numpy as np

class FeaturesSequence(keras.utils.Sequence):
    def __init__(self, x_sequence, x_meta, y_set, 
                         batch_size, sample_length, feature_dimensions, 
                         duke_unique, dnase):

        assert len(x_sequence) == len(x_meta) == len(y_set)
        self.x_sequence, self.x_meta, self.y = x_sequence, x_meta, y_set
        self.y_are_labels = type(y_set[0])==str
        logging.info('Found y are labels; using OBIE')
        if self.y_are_labels:
            self.label2onehot = {'O':np.array([1,0,0,0]),'B':np.array([0,1,0,0]),'I':np.array([0.,0.,1.,0.]),'E':np.array([0.,0.,0.,1.]),}
        self.sample_length=sample_length
        self.n_samples = len(self.x_meta)
        self.feature_dimensions=feature_dimensions
        self.batch_size = batch_size
        self.onehot = {'A':np.array([1,0,0,0]),'C':np.array([0,1,0,0]),'G':np.array([0.,0.,1.,0.]),'T':np.array([0.,0.,0.,1.]),}
        self.duke_unique = duke_unique
        self.dnase = dnase
            
    def __len__(self):
        return int(np.ceil(self.n_samples / float(self.batch_size))) 

    def __getitem__(self, idx):
        # setup y
        batch_y = []
        for i in range(self.batch_size):
            if self.y_are_labels:
                labels = self.y[(i+idx*self.batch_size)%self.n_samples]
                labels_onehot = np.empty((len(labels), 4))
                for j in range(len(labels)):
                    labels_onehot[j, :] = self.label2onehot[labels[j]]
                batch_y.append(labels_onehot)
            else:
                batch_y.append(self.y[(i+idx*self.batch_size)%self.n_samples])

        # setup x
        batch_x = []
        for i in range(self.batch_size):
            seq, (chromosome, (start,stop)) = self.x_sequence[(i+idx*self.batch_size)%self.n_samples], self.x_meta[(i+idx*self.batch_size)%self.n_samples]
            # this part is potentially very slow #
            uniqueness = np.array(self.duke_unique(chromosome, start, stop))
            openness = np.array(self.dnase(chromosome, start, stop))
            uniqueness[np.isnan(uniqueness)] = 0
            openness[np.isnan(openness)] = 0
            # # # # # # # # # # # # # # # # # # #
            features = np.empty((self.sample_length, self.feature_dimensions))
            for j in range(self.sample_length):
                features[j,:] = (np.concatenate((self.onehot[seq[j]], [openness[j]], [uniqueness[j]])))
            batch_x.append(features)
            
        # setup sample weights
#         batch_weight = []
#         for i in range(self.batch_size):
#             batch_weight.append(self.sample_weights[(i+idx*self.batch_size)%self.n_samples])
        return np.array(batch_x), np.array(batch_y) #, np.array(batch_weight)

In [21]:
import keras
import numpy as np

class TrainingFeaturesSequence(keras.utils.Sequence):
    def __init__(self, x_sequence, x_meta, y_set, 
                         batch_size, sample_length, feature_dimensions, 
                         duke_unique, dnase, sample_weights=None):

        assert len(x_sequence) == len(x_meta) == len(y_set)
        self.x_sequence_pos, self.x_meta_pos, self.y_pos = x_sequence[0], x_meta[0], y_set[0]
        self.x_sequence_neg, self.x_meta_neg = x_sequence[1], x_meta[1]
        self.y_are_labels = type(y_set[0][0])==str
        logging.info('Found y are labels; using OBIE')
        if self.y_are_labels:
            self.label2onehot = {'O':np.array([1,0,0,0]),'B':np.array([0,1,0,0]),'I':np.array([0.,0.,1.,0.]),'E':np.array([0.,0.,0.,1.]),}
        self.sample_length=sample_length
        self.n_samples_pos, self.n_samples_neg = len(self.x_meta_pos), len(self.x_meta_neg)
        self.feature_dimensions=feature_dimensions
        self.batch_size = batch_size
        self.onehot = {'A':np.array([1,0,0,0]),'C':np.array([0,1,0,0]),'G':np.array([0.,0.,1.,0.]),'T':np.array([0.,0.,0.,1.]),}
        self.duke_unique = duke_unique
        self.dnase = dnase
            
    def __len__(self):
        return int(np.ceil(self.n_samples_pos / float(self.batch_size))) 

    def __getitem__(self, idx):
        # setup y
        half_batch_size = int(self.batch_size/2)
        batch_y = []
        for i in range(self.batch_size):
            if i<half_batch_size:
                # work on positive samples
                if self.y_are_labels:
                    labels = self.y_pos[(i+idx*half_batch_size)%self.n_samples_pos]
                    labels_onehot = np.empty((len(labels), 4))
                    for j in range(len(labels)):
                        labels_onehot[j, :] = self.label2onehot[labels[j]]
                    batch_y.append(labels_onehot)
                else:
                    batch_y.append(1)
            else:
                if self.y_are_labels:
#                     labels = self.y_neg[(i+idx*self.batch_size/2)%self.n_samples_neg]
                    labels_onehot = np.empty((self.sample_length, 4))
                    for j in range(self.sample_length):
                        labels_onehot[j, :] = self.label2onehot['O']
                    batch_y.append(labels_onehot)
                else:
                    batch_y.append(0)

        # setup x
        batch_x = []
        for i in range(self.batch_size):
            if i<half_batch_size:
                seq, (chromosome, (start,stop)) = self.x_sequence_pos[(i+idx*half_batch_size)%self.n_samples_pos], self.x_meta_pos[(i+idx*half_batch_size)%self.n_samples_pos]
                # this part is potentially very slow #
                uniqueness = np.array(self.duke_unique(chromosome, start, stop))
                openness = np.array(self.dnase(chromosome, start, stop))
                uniqueness[np.isnan(uniqueness)] = 0
                openness[np.isnan(openness)] = 0
                # # # # # # # # # # # # # # # # # # #
                features = np.empty((self.sample_length, self.feature_dimensions))
                for j in range(self.sample_length):
                    features[j,:] = (np.concatenate((self.onehot[seq[j]], [openness[j]], [uniqueness[j]])))
                batch_x.append(features)
            else:
                seq, (chromosome, (start,stop)) = self.x_sequence_neg[(i+idx*half_batch_size-half_batch_size)%self.n_samples_neg], self.x_meta_neg[(i+idx*half_batch_size-half_batch_size)%self.n_samples_neg]
                # this part is potentially very slow #
                uniqueness = np.array(self.duke_unique(chromosome, start, stop))
                openness = np.array(self.dnase(chromosome, start, stop))
                uniqueness[np.isnan(uniqueness)] = 0
                openness[np.isnan(openness)] = 0
                # # # # # # # # # # # # # # # # # # #
                features = np.empty((self.sample_length, self.feature_dimensions))
                for j in range(self.sample_length):
                    features[j,:] = (np.concatenate((self.onehot[seq[j]], [openness[j]], [uniqueness[j]])))
                batch_x.append(features)
            
        # setup sample weights
#         batch_weight = []
#         for i in range(self.batch_size):
#             batch_weight.append(self.sample_weights[(i+idx*self.batch_size)%self.n_samples])
        return np.array(batch_x), np.array(batch_y) #,np.array(batch_weight)

In [22]:
model_paths = []
for d in os.listdir('output models/'):
    print('Looking in '+d)
    subdir = os.path.join('output models',d)
    if os.path.isdir(subdir):
        print('>Looking in sub '+subdir)
        for model_d in os.listdir(subdir):
            path = os.path.join(subdir,model_d)
            tokens = model_d.split('_')
            tf = tokens[0]
            model_type = '_'.join(tokens[1:-2])
            
            saved_points = list(filter(lambda f: f.endswith('hdf5'),os.listdir(path)))
            x = [pt.split('-') for pt in saved_points]
            x = sorted(saved_points, key= lambda pt : pt.split('-')[1], reverse=True)
            try:
                logging.debug("Found model in %s, best checkpoint %s"%(model_d, x[0]))
                model_paths.append(os.path.join('output models', d, model_d, x[0]))
            except:
                pass
    else:
        print('> '+d+' is not a directory')
 

Looking in CTCF_outputs
>Looking in sub output models\CTCF_outputs


DEBUG:root:Found model in CTCF_bigru_crf_model_outputs, best checkpoint CTCF_bigru_crf-0004-0.95.hdf5
DEBUG:root:Found model in CTCF_deepbind_model_outputs, best checkpoint CTCF_deepbind-0013-0.91.hdf5
DEBUG:root:Found model in CTCF_deepbind_plus_model_outputs, best checkpoint CTCF_deepbind_plus-0005-0.92.hdf5
DEBUG:root:Found model in CTCF_factor_net_model_outputs, best checkpoint CTCF_factor_net-0041-0.96.hdf5
DEBUG:root:Found model in CTCF_s_bigru_crf_model_outputs, best checkpoint CTCF_s_bigru_crf-0005-0.93.hdf5
DEBUG:root:Found model in CTCF_s_bilstm_crf_model_outputs, best checkpoint CTCF_s_bilstm_crf-0055-0.93.hdf5
DEBUG:root:Found model in CTCF_s_cnn_bigru_crf_model_outputs, best checkpoint CTCF_s_cnn_bigru_crf-0067-0.96.hdf5


Looking in E2F1_outputs
>Looking in sub output models\E2F1_outputs


DEBUG:root:Found model in E2F1_bigru_crf_model_outputs, best checkpoint E2F1_bigru_crf-0011-0.97.hdf5
DEBUG:root:Found model in E2F1_factor_net_model_outputs, best checkpoint E2F1_factor_net-0021-0.96.hdf5
DEBUG:root:Found model in E2F1_s_bigru_crf_model_outputs, best checkpoint E2F1_s_bigru_crf-0200-0.96.hdf5
DEBUG:root:Found model in E2F1_s_bilstm_crf_model_outputs, best checkpoint E2F1_s_bilstm_crf-0002-0.96.hdf5
DEBUG:root:Found model in E2F1_s_cnn_bigru_crf_model_outputs, best checkpoint E2F1_s_cnn_bigru_crf-0002-0.08.hdf5


Looking in EGR1_outputs
>Looking in sub output models\EGR1_outputs


...

DEBUG:root:Found model in MAX_bigru_crf_model_outputs, best checkpoint MAX_bigru_crf-0035-0.96.hdf5
DEBUG:root:Found model in MAX_deepbind_model_outputs, best checkpoint MAX_deepbind-0005-0.95.hdf5
DEBUG:root:Found model in MAX_deepbind_plus_model_outputs, best checkpoint MAX_deepbind_plus-0011-0.95.hdf5
DEBUG:root:Found model in MAX_factor_net_model_outputs, best checkpoint MAX_factor_net-0002-0.95.hdf5
DEBUG:root:Found model in MAX_s_bigru_crf_model_outputs, best checkpoint MAX_s_bigru_crf-0019-0.96.hdf5
DEBUG:root:Found model in MAX_s_cnn_bigru_crf_model_outputs, best checkpoint MAX_s_cnn_bigru_crf-0001-0.95.hdf5


Looking in REST_outputs
>Looking in sub output models\REST_outputs


DEBUG:root:Found model in REST_bigru_crf_model_outputs, best checkpoint REST_bigru_crf-0036-0.96.hdf5
DEBUG:root:Found model in REST_factor_net_model_outputs, best checkpoint REST_factor_net-0001-0.94.hdf5
DEBUG:root:Found model in REST_s_bigru_crf_model_outputs, best checkpoint REST_s_bigru_crf-0030-0.95.hdf5
DEBUG:root:Found model in REST_s_bilstm_crf_model_outputs, best checkpoint REST_s_bilstm_crf-0045-0.96.hdf5
DEBUG:root:Found model in REST_s_cnn_bigru_crf_model_outputs, best checkpoint REST_s_cnn_bigru_crf-0001-0.95.hdf5


Looking in TAF1_outputs
>Looking in sub output models\TAF1_outputs


DEBUG:root:Found model in TAF1_bigru_crf_model_outputs, best checkpoint TAF1_bigru_crf-0029-0.96.hdf5
DEBUG:root:Found model in TAF1_factor_net_model_outputs, best checkpoint TAF1_factor_net-0001-0.96.hdf5
DEBUG:root:Found model in TAF1_s_bigru_crf_model_outputs, best checkpoint TAF1_s_bigru_crf-0018-0.96.hdf5
DEBUG:root:Found model in TAF1_s_cnn_bigru_crf_model_outputs, best checkpoint TAF1_s_cnn_bigru_crf-0002-0.95.hdf5


In [23]:
from keras.layers import Dense
from keras.models import Model

def create_binary_model(model_path):
    model = load_model(model_paths[0], custom_objects={'ReverseComplementLayer': ReverseComplementLayer, 
                                                             "ClassWrapper": ClassWrapper ,
                                                             "CRF_ext": ClassWrapper, "loss": loss, "accuracy":accuracy,
                                                             "viterbi_precision":viterbi_precision, "f1":f1,
                                                             "recall":recall, "precision":precision})
    model.trainable = False
    x = model.output
    x = Dense(1, activation='sigmoid')(x)

    layered_model = Model(inputs=model.input, outputs=x)
    layered_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return layered_model

In [24]:
model_path = list(filter(lambda p: 'bigru' in p and 'TAF1' in p, model_paths))[0]

In [25]:
model_path

'output models\\TAF1_outputs\\TAF1_bigru_crf_model_outputs\\TAF1_bigru_crf-0029-0.96.hdf5'

In [26]:
tf = model_path[len('output models\\'):].split('_')[0]

In [27]:
model_type = '_'.join(model_path.split('\\')[2].split('_')[1:-2])

In [28]:
label_file = '%s._.labels.tsv' % (tf.upper())

In [29]:
import re
peakfile_regex = re.compile(r'chipseq\.\w+\.%s\..*\.\_\.narrowpeak'%tf, re.IGNORECASE)
peakfile = list(filter(lambda f: peakfile_regex.match(f), os.listdir('.')))[0]

In [30]:
filter_file='total_regions.blacklistfiltered.merged.bed'

In [31]:
reduce_negative_samples=False

# Construct datamanager

In [32]:
def auROC(labels, predictions):
    return roc_auc_score(labels, predictions)

def auPRC(labels, predictions):
    precision, recall = precision_recall_curve(labels, predictions)[:2]
    return auc(recall, precision)

def recall_at_precision(labels, predictions, precision_at):
    threshold = 1.0-precision_at
    precision, recall = precision_recall_curve(labels, predictions)[:2]
    return 100 * recall[np.searchsorted(precision - threshold, 0)]

In [33]:
binary_model = None
if peakfile and os.path.exists(label_file):
    
    logging.info('Working on %s'%peakfile)
    dm = DataManager(label_file, peakfile, output_dir='datasets', 
                        move_finished_src='finished_peakfiles', 
                        filter_file=filter_file,
                        reduce_negative_samples=reduce_negative_samples, # check_set_ratio=9, # use for reduce_negative_samples=True
                        )
    dm.samples # make sure samples are already created
    generators = dm.create_datagen_from_samples(useDNAshapeR=False)
    train_labels_sequencer, train_cls_sequencer = generators[:2]
    valid_x, valid_label, valid_seq, valid_cls, valid_labels_sequencer, valid_cls_sequencer = generators[2:8]
    test_x, test_label, test_seq, test_cls, test_labels_sequencer, test_cls_sequencer = generators[8:]
    
    binary_model = create_binary_model(model_path)
    batch_n, epoch_n = 512, 200
    train_len = dm.dataset2counts_pos['train'] * 2
    test_len = dm.dataset2counts_pos['test'] + dm.dataset2counts_neg['test']

    train_steps = train_len//batch_n
    validation_steps = int((train_len/4)//batch_n)
    test_steps = test_len//batch_n

    train_generator = train_cls_sequencer(batch_n)
    validation_generator = valid_cls_sequencer(batch_n)

    history = binary_model.fit_generator(train_cls_sequencer,
                                         steps_per_epoch= train_steps,
                                         epochs=epoch_n,
                                         callbacks=[],
                                         validation_data=valid_cls_sequencer,
                                         validation_steps=validation_steps,
                                         verbose=1)
    test_pred = bin_model.predict_generator(test_x(batch_n), steps=test_steps, verbose=1)
    test_pred = list(test_pred[:,0])
    import itertools
    test_real = [x for batch in itertools.islice(test_cls(batch_n),test_steps) for x in batch ]

    # auROC,  auPRC, recalls
    logging.info('Calculating auROC')
    auroc = auROC(test_real, test_pred)
    logging.info('auROC: {}'.format(auroc))
    logging.info('Calculating auPRC')
    auprc = auPRC(test_real, test_pred)
    logging.info('auPRC: {}'.format(auprc))
    logging.info('Calculating recall @ precisions')
    re5, re10, re25, re50 = [recall_at_precision(test_real, test_pred, precision) for precision in [0.05,0.1,0.25,0.5]]
    # confusion matrix 
    confusion = confusion_matrix(test_real, np.round(test_pred))

    out = 'auROC: {} auPRC {} re@5/10/25/50: {:>7.5}/{:>7.5}/{:>7.5}/{:>7.5}'.format(auroc, auprc, re5, re10, re25, re50)
    with open(tf+'_'+model_type+'_remodeled_output', 'w') as write_to:
        write_to.write(out)
else:
    logging.info(peakfile + " " + str(os.path.exists(label_file)))

INFO:root:Working on chipseq.liver.taf1.conservative._.narrowpeak
INFO:root:Loading positive/negative samples from disk
INFO:root:Done loading samples from disk
INFO:root:Reading static data
INFO:root:Loaded training/valid/test of sizes 209314/65140/7902816
INFO:root:Creating generators
INFO:root:Creating feature generators
INFO:root:Calculating sample weights
INFO:root:Creating features sequencers
INFO:root:Creating value generators
INFO:root:The function values_gen took     0.0000s
INFO:root:The function values_gen took     0.0000s
INFO:root:The function values_gen took     0.0000s
INFO:root:The function values_gen took     0.0000s
INFO:root:The function values_gen took     0.0000s
INFO:root:The function values_gen took     0.0000s
INFO:root:Found y are labels; using OBIE
INFO:root:Found y are labels; using OBIE


ValueError: An operation has `None` for gradient. Please make sure that all of your ops have a gradient defined (i.e. are differentiable). Common ops without gradient: K.argmax, K.round, K.eval.